In [ ]:
import tensorflow as tf#1.x vesion
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout,Conv2D, MaxPooling3D, Conv3D, MaxPooling2D, TimeDistributed,LSTM
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.applications import ResNet50,DenseNet121,EfficientNetB0
# https://blog.csdn.net/heiheiya/article/details/102776353
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
"""
#tensorflow 2.x
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
print(gpus, cpus)

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
"""
#tensorflow 1.x
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)


import matplotlib.pyplot as plt

import numpy as np


In [ ]:
num_classes = 4
img_size = (150,150)
X = np.load("../Data/Skin_Cancer/X_Skin_ISIC_Multi_150.npy")
Y = np.load("../Data/Skin_Cancer/Y_Skin_ISIC_Multi_150.npy")

In [ ]:

index = [i for i in range(X.shape[0])]
np.random.shuffle(index) 


X2 = X[index]/255.0
Y2 = Y[index]

In [ ]:
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
label_encode = LabelEncoder()
Y3 = label_encode.fit_transform(Y2)

In [ ]:
def build_model():
    inputs = layers.Input(shape=(150, 150, 3))

    model = ResNet50(include_top=False, input_tensor=inputs, weights=None)
    #model = DenseNet121(include_top=False, input_tensor=inputs, weights = None)
    #model = EfficientNetB0(include_top=False, input_tensor=inputs, weights=None)


    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    
    x = layers.BatchNormalization()(x)


    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile

    model = tf.keras.Model(inputs, outputs, name="ResNet50")
    optimizer = tf.keras.optimizers.Adam(lr=1e-3)
    model.compile(
        optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["acc"]
    )
    return model

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score,f1_score,accuracy_score,confusion_matrix,precision_score,recall_score
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.1,
                              patience=10)
earlystop = EarlyStopping(monitor='val_acc', patience=15)

In [ ]:
seed = 101
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvacc = []
cvauc = []
cvf1 = []
i = 1

for train, test in kfold.split(X2,Y2):

    model = build_model()
    # checkpoint
    filepath = "../Data/models/150_cnn{0}_skin_multi_resnet50.h5".format(str(i))
    if os.path.exists(filepath):
        os.remove(filepath)

    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint,earlystop]

    model.fit(X2[train], Y3[train],validation_split=0.2, epochs=300, batch_size=32, verbose=1,shuffle=True,callbacks = callbacks_list)
    # evaluate the model
    model.load_weights(filepath)
    
    y_pred = model.predict(X2[test])
    acc = accuracy_score(Y3[test],[np.argmax(x) for x in y_pred])
    f1 = f1_score(Y3[test],[np.argmax(x) for x in y_pred],average='macro')
 
    auc = roc_auc_score(Y3[test],y_pred,multi_class='ovo')
    precisoin = precision_score(Y3[test],[np.argmax(x) for x in y_pred],average='macro')
    recall = recall_score(Y3[test],[np.argmax(x) for x in y_pred],average='macro')
    
    print("acc: %.2f%%" % (acc))
    print("auc: %.2f"%(auc))
    print("f1: %.2f"%(f1))
    print('precision:%.2f'%(precision))
    print('recall:%.2f'%(recall))
    print("confusion matrix:",confusion_matrix(Y3[test],[np.argmax(x) for x in y_pred]))
    
    cvacc.append(acc)
    cvauc.append(auc)
    cvf1.append(f1)
    cvprecision.append(precision)
    cvrecall.append(recall)

    i+=1
print("mean acc %.2f%% (+/- %.2f%%)" % (np.mean(cvacc), np.std(cvacc)))
print("mean auc %.2f%% (+/- %.2f%%)" % (np.mean(cvauc), np.std(cvauc)))
print("mean f1 %.2f%% (+/- %.2f%%)" % (np.mean(cvf1), np.std(cvf1)))
print("mean precision %.2f%% (+/- %.2f%%)" % (np.mean(cvprecision), np.std(cvprecision)))
print("mean recall %.2f%% (+/- %.2f%%)" % (np.mean(cvrecall), np.std(cvrecall)))

In [ ]:
cvacc

In [ ]:
np.mean(cvacc)

In [ ]:
cvauc

In [ ]:
np.mean(cvauc)

In [ ]:
cvf1

In [ ]:
np.mean(cvf1)